In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Getting helper function

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-27 05:47:00--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-27 05:47:00 (54.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-06-27 05:47:11--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 108.177.98.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2023-06-27 05:47:11 (90.2 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



### Visualizing dataset

In [5]:
import pandas as pd

In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [10]:
print(f"Total training samples: {len(train_df)}")
print(f"Total testing samples: {len(test_df)}")
print(f"Total samples: {len(train_df) + len(test_df)}")

Total training samples: 7613
Total testing samples: 3263
Total samples: 10876


In [11]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [12]:
import random

In [13]:
random_index = random.randint(0, len(train_df)-5)

for index in range(random_index, random_index+5):
    print(f'Target: {train_df["target"][index]}')
    print(f'Text:\n{train_df["text"][index]}')
    print("\n")

Target: 1
Text:
2pcs 18W CREE Led Work Light  Offroad Lamp Car Truck Boat Mining 4WD FLOOD BEAM - Full reaÛ_ http://t.co/ApWXS5Mm44 http://t.co/DS76loZLSu


Target: 0
Text:
@hellotybeeren cue the flood of people 'ironically' calling you that


Target: 0
Text:
Motors Hot Deals #452 &gt;&gt; http://t.co/ED32PBviO7 10x 27W 12V 24V LED Work Light FLOOD Lamp Tractor Truck SUV UTV AÛ_ http://t.co/IfM6v6480P


Target: 0
Text:
85V-265V 10W LED Warm White Light Motion Sensor Outdoor Flood Light PIR Lamp AUC http://t.co/NJVPXzMj5V http://t.co/Ijd7WzV5t9


Target: 1
Text:
iembot_hfo : At 10:00 AM 2 NNW Hana [Maui Co HI] COUNTY OFFICIAL reports COASTAL FLOOD #Û_ http://t.co/Gg0dZSvBZ7) http://t.co/kBe91aRCdw




### Spliting Data

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42)

In [16]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [17]:
temp = pd.DataFrame(data=train_sentences[:10], columns=["text"])
temp["target"] = train_labels[:10]
temp

,text,target
0,@mogacola @zamtriossu i screamed after hitting...,0
1,Imagine getting flattened by Kurt Zouma,0
2,@Gurmeetramrahim #MSGDoing111WelfareWorks Gree...,1
3,@shakjn @C7 @Magnums im shaking in fear he's g...,0
4,Somehow find you and I collide http://t.co/Ee8...,0
5,@EvaHanderek @MarleyKnysh great times until th...,1
6,destroy the free fandom honestly,1
7,Weapons stolen from National Guard Armory in N...,0
8,@wfaaweather Pete when will the heat wave pass...,1
9,Patient-reported outcomes in long-term survivo...,1


### Text vectorization (tokenization)

In [18]:
from tensorflow.keras.layers import TextVectorization

In [19]:
avg_sentence_length = round(sum([len(sentence.split()) for sentence in train_sentences]) / len(train_sentences))
avg_sentence_length

15

In [20]:
# this method's every parameter is set by default, what we need. Just need to change below two params.
text_vectorizer = TextVectorization(max_tokens=10000,
                                    output_sequence_length=avg_sentence_length)

In [21]:
text_vectorizer.adapt(train_sentences)

In [22]:
import random

In [23]:
random_sentence = random.choice(train_sentences)
print(f"Before tokenization: {random_sentence}")
print(f"After tokenization: {text_vectorizer([random_sentence])}")

Before tokenization: Wildfire near Columbia River town is 50 percent contained http://t.co/gzGpWSqyMW #FireNews #WA http://t.co/ay49MTYyL8
After tokenization: [[ 146  217 3988  499  801    9  717 1494 3976    1    1 1739    1    0
     0]]


In [24]:
print(f"5 most common words: {text_vectorizer.get_vocabulary()[:5]}")
print(f"5 least common words: {text_vectorizer.get_vocabulary()[-5:]}")

5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Embedding

In [25]:
from tensorflow.keras.layers import Embedding

In [26]:
embedding = Embedding(input_dim=len(text_vectorizer.get_vocabulary()),
                      output_dim=128,
                      input_length=avg_sentence_length)

In [27]:
random_sentence = random.choice(train_sentences)
print(f"Sentence before embedding: {random_sentence}")
embedding(text_vectorizer([random_sentence]))

Sentence before embedding: Someone walk with me to DQ ??
I wanna Butterfinger Blizzard so bad??


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.0142696 ,  0.0483819 , -0.02450012, ..., -0.0085609 ,
          0.01856995,  0.02399036],
        [ 0.03930053, -0.01743497,  0.03733708, ..., -0.00906906,
         -0.00273889, -0.00146256],
        [-0.04356396, -0.04701489,  0.02328009, ...,  0.02071948,
         -0.02614009, -0.01733396],
        ...,
        [-0.00760078,  0.02995101,  0.03955982, ..., -0.02473738,
          0.03301575,  0.00669945],
        [-0.00760078,  0.02995101,  0.03955982, ..., -0.02473738,
          0.03301575,  0.00669945],
        [-0.00760078,  0.02995101,  0.03955982, ..., -0.02473738,
          0.03301575,  0.00669945]]], dtype=float32)>

**Note:** If we see the shape of the embedded sentence, we will notice it is (1, 15, 128)<br>
which means from every sentence we will take 15 words and each word will be represented with 128 shape matrix.

### `model_0`: Baseline

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [29]:
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("multinomial", MultinomialNB())
])

In [30]:
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('multinomial', MultinomialNB())])

### Evaluation function

In [31]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [32]:
def calculate_results(y_true, y_pred):
    """
    returns a dictionary of accuracy_score, precission, recall and f1_score
    """
    precission, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    acc_score = accuracy_score(y_true, y_pred) * 100
    
    results = {
        "accuracy_score": acc_score,
        "precission": precission,
        "recall": recall,
        "f1_score": f1_score
    }
    return results

In [33]:
baseline_results = calculate_results(val_labels, model_0.predict(val_sentences))
baseline_results

{'accuracy_score': 79.26509186351706,
 'precission': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

### `model_1`: Simple dense model

In [34]:
import tensorflow as tf
from tensorflow.keras import layers

In [35]:
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.GlobalAveragePooling1D()(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [36]:
model_1.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [37]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [38]:
SAVE_DIR = "model_logs"

In [39]:
history_1 = model_1.fit(train_sentences, 
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                               experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230627-054716
Epoch 1/5
215/215 [==============================] - 22s 80ms/step - loss: 0.6384 - accuracy: 0.6456 - val_loss: 0.5739 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4693 - accuracy: 0.8270 - val_loss: 0.4769 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3418 - accuracy: 0.8740 - val_loss: 0.4552 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2595 - accuracy: 0.9069 - val_loss: 0.4563 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1996 - accuracy: 0.9318 - val_loss: 0.4695 - val_accuracy: 0.7900


In [40]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.47314653],
       [0.89300686],
       [0.9934016 ],
       [0.05569652],
       [0.3453293 ],
       [0.96041924],
       [0.92070585],
       [0.98776287],
       [0.916852  ],
       [0.10766154]], dtype=float32)

In [41]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [42]:
model_1_result = calculate_results(y_true=val_labels,
                                 y_pred=model_1_preds)
model_1_result

{'accuracy_score': 79.00262467191601,
 'precission': 0.795059849830577,
 'recall': 0.7900262467191601,
 'f1_score': 0.7870730838048355}

### `model_2`: LSTM

In [43]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(units=64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

In [44]:
model_2.compile(loss="binary_crossentropy", 
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [45]:
history_2 = model_2.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                              "LSTM_model")])

Saving TensorBoard log files to: model_logs/LSTM_model/20230627-054749
Epoch 1/5
215/215 [==============================] - 20s 68ms/step - loss: 0.3686 - accuracy: 0.8343 - val_loss: 0.4949 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.2414 - accuracy: 0.9044 - val_loss: 0.5493 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.1772 - accuracy: 0.9314 - val_loss: 0.5968 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.1340 - accuracy: 0.9527 - val_loss: 0.6652 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0974 - accuracy: 0.9628 - val_loss: 0.8904 - val_accuracy: 0.7585


In [46]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10] 

24/24 [==============================] - 1s 6ms/step


array([[0.00801453],
       [0.3973338 ],
       [0.9979533 ],
       [0.01835955],
       [0.00115442],
       [0.99961925],
       [0.5473311 ],
       [0.99986076],
       [0.9998047 ],
       [0.4174238 ]], dtype=float32)

In [47]:
model_2_pred = tf.squeeze(tf.round(model_1_pred_probs))
model_2_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [48]:
model_2_result = calculate_results(y_true=val_labels,
                                  y_pred=model_2_pred)
model_2_result

{'accuracy_score': 79.00262467191601,
 'precission': 0.795059849830577,
 'recall': 0.7900262467191601,
 'f1_score': 0.7870730838048355}

In [49]:
baseline_results

{'accuracy_score': 79.26509186351706,
 'precission': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

### `model_3': GRU

In [50]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
x = layers.GRU(units=64, return_sequences=True)(x)
# print(x.shape)
x = layers.GRU(units=64)(x)
# print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="GRU_model")

In [51]:
model_3.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [52]:
history_3 = model_3.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                             "GRU_model")])

Saving TensorBoard log files to: model_logs/GRU_model/20230627-054835
Epoch 1/5
215/215 [==============================] - 17s 60ms/step - loss: 0.1892 - accuracy: 0.9215 - val_loss: 0.5847 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0991 - accuracy: 0.9657 - val_loss: 0.6225 - val_accuracy: 0.7612
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0771 - accuracy: 0.9724 - val_loss: 0.7391 - val_accuracy: 0.7717
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0633 - accuracy: 0.9720 - val_loss: 1.0822 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0508 - accuracy: 0.9774 - val_loss: 1.0544 - val_accuracy: 0.7743


In [53]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:5]

24/24 [==============================] - 1s 4ms/step


array([[9.7715342e-04],
       [6.5716738e-01],
       [9.9860722e-01],
       [3.7403792e-02],
       [5.6305376e-04]], dtype=float32)

In [54]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [55]:
model_3_result = calculate_results(y_true=val_labels,
                                  y_pred=model_3_pred)
model_3_result

{'accuracy_score': 77.42782152230971,
 'precission': 0.782466266919887,
 'recall': 0.7742782152230971,
 'f1_score': 0.769750137241332}

### `model_4`: Bidirectional

In [56]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.Bidirectional(layers.GRU(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
print(x.shape)
# x = layers.Bidirectional(64)(x)
print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="bidirectional_model")

(None, 15, 128)
(None, 128)
(None, 128)


In [57]:
model_4.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [58]:
history_4 = model_4.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                             "bidirectional_model")])

Saving TensorBoard log files to: model_logs/bidirectional_model/20230627-054909
Epoch 1/5
215/215 [==============================] - 24s 70ms/step - loss: 0.1322 - accuracy: 0.9491 - val_loss: 0.7887 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 6s 30ms/step - loss: 0.0584 - accuracy: 0.9753 - val_loss: 1.2794 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0496 - accuracy: 0.9785 - val_loss: 1.4394 - val_accuracy: 0.7572
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0452 - accuracy: 0.9797 - val_loss: 1.3586 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0429 - accuracy: 0.9810 - val_loss: 1.7083 - val_accuracy: 0.7559


In [59]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:5]

24/24 [==============================] - 2s 6ms/step


array([[3.0755004e-05],
       [8.1351328e-01],
       [9.9991560e-01],
       [1.8012258e-01],
       [2.4238607e-05]], dtype=float32)

In [60]:
model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [61]:
model_4_result = calculate_results(val_labels, model_4_pred)
model_4_result

{'accuracy_score': 75.59055118110236,
 'precission': 0.7571361526829015,
 'recall': 0.7559055118110236,
 'f1_score': 0.7536918613918199}

### `model_5`: Convolutional network

In [62]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
print(x.shape)
x = layers.GlobalMaxPool1D()(x)
print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Conv1D")

(None, 15, 128)
(None, 11, 32)
(None, 32)


In [63]:
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d (Conv1D)             (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 1)              

In [64]:
model_5.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [65]:
history_5 = model_5.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                             "model_5_Conv1D")])

Saving TensorBoard log files to: model_logs/model_5_Conv1D/20230627-054953
Epoch 1/5
215/215 [==============================] - 18s 60ms/step - loss: 0.1852 - accuracy: 0.9372 - val_loss: 0.7554 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0816 - accuracy: 0.9715 - val_loss: 0.9190 - val_accuracy: 0.7546
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0650 - accuracy: 0.9758 - val_loss: 0.9916 - val_accuracy: 0.7480
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0555 - accuracy: 0.9784 - val_loss: 1.0798 - val_accuracy: 0.7467
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.0534 - accuracy: 0.9801 - val_loss: 1.0782 - val_accuracy: 0.7454


In [66]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:5]

24/24 [==============================] - 0s 3ms/step


array([[9.3792766e-02],
       [8.8394451e-01],
       [9.9933064e-01],
       [3.3343669e-02],
       [6.5709783e-06]], dtype=float32)

In [67]:
model_5_pred = tf.squeeze(tf.round(model_5_pred_probs))
model_5_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [68]:
model_5_result = calculate_results(val_labels, model_5_pred)
model_5_result

{'accuracy_score': 74.54068241469817,
 'precission': 0.745811509244164,
 'recall': 0.7454068241469817,
 'f1_score': 0.743511039286887}

### `model_6`: Transfer learning

In [69]:
import tensorflow_hub as hub

In [80]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        trainable=False,
#                                         input_shape=[],
#                                         dtype="string"
                                       )

In [83]:
# outputs =  layers.Dense(1, activation="sigmoid")(sentence_encoder_layer)
# model_6 = tf.keras.Model(sentence_encoder_layer, outputs)

In [81]:
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

In [85]:
model_6.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 512)               256797824 
                                                                 
 dense_10 (Dense)            (None, 64)                32832     
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [82]:
model_6.compile(loss="binary_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

In [84]:
history_6 = model_6.fit(train_sentences,
                       train_labels,
                       epochs=5,
                       validation_data=(val_sentences, val_labels),
                       callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                             "transfer_learning")])

Saving TensorBoard log files to: model_logs/transfer_learning/20230627-060834
Epoch 1/5
215/215 [==============================] - 10s 19ms/step - loss: 0.5039 - accuracy: 0.7879 - val_loss: 0.4518 - val_accuracy: 0.7992
Epoch 2/5
215/215 [==============================] - 4s 19ms/step - loss: 0.4141 - accuracy: 0.8171 - val_loss: 0.4392 - val_accuracy: 0.8071
Epoch 3/5
215/215 [==============================] - 5s 23ms/step - loss: 0.3998 - accuracy: 0.8260 - val_loss: 0.4335 - val_accuracy: 0.8084
Epoch 4/5
215/215 [==============================] - 4s 16ms/step - loss: 0.3917 - accuracy: 0.8276 - val_loss: 0.4292 - val_accuracy: 0.8150
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.3857 - accuracy: 0.8301 - val_loss: 0.4260 - val_accuracy: 0.8136


In [86]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:5]

24/24 [==============================] - 1s 12ms/step


array([[0.1849886 ],
       [0.78987235],
       [0.98930883],
       [0.20156652],
       [0.77319396]], dtype=float32)

In [87]:
model_6_pred = tf.squeeze(tf.round(model_6_pred_probs))
model_6_pred[:5]

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 1.], dtype=float32)>

In [88]:
model_6_result = calculate_results(val_labels, model_6_pred)
model_6_result

{'accuracy_score': 81.36482939632546,
 'precission': 0.8147394195152357,
 'recall': 0.8136482939632546,
 'f1_score': 0.8125379768469353}

In [89]:
model_1_result

{'accuracy_score': 79.00262467191601,
 'precission': 0.795059849830577,
 'recall': 0.7900262467191601,
 'f1_score': 0.7870730838048355}